In [2]:
#importing packages
import time
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random
import requests
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from functools import reduce
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

In [3]:
#mounting to google drive for live in-game odd changes
from google.colab import drive

drive.mount('/content/drive/')
PATH = "/content/drive/My Drive/"

Mounted at /content/drive/


Saving the initial lines (do this before games start)

In [4]:
#imports
#RUN ONLY 1 TIME: collects 'initial odds': therefore all differences will be 0 at time 0, as time goes on since running (and games start), the odds change

#function specific to json ex: https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json
def parse_data(jsooooon):
    results = pd.DataFrame()
    for ayo in jsooooon['events']:
        money = json_normalize(ayo).drop('markets',axis=1)
        for bandos in ayo['markets']:
            gambling = json_normalize(bandos).drop('selections',axis=1)
            gambling.columns = [str(col) + '.markets' for col in gambling.columns]
            for theta in bandos['selections']:
                yeye = json_normalize(theta)
                yeye.columns = [str(col) + '.selections' for col in yeye.columns]
                yoyo = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), [money, gambling, yeye])
                results = results.append(yoyo, sort=True).reset_index(drop=True)
    return results

#getting jsons, nfl doesnt work
nfl = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/57494.3.json').json()
nba = requests.get('https://sportsbook.fanduel.com/cache/psmg/UK/55978.3.json').json()
bball = requests.get('https://sportsbook.fanduel.com/cache/psmg/US/53474.3.json').json()


#setting up df
cbb = parse_data(bball)
cb = pd.DataFrame()
cbb_ = cbb[cbb['name.markets']=='Spread Betting']
cb[['lines','team','away_t','home_t','start_time']] = cbb_[['currenthandicap.selections','name.selections','participantname_away','participantname_home','tsstart']]

away = cb[cb['team']==cb['away_t']]
away['lines'] = away['lines'].astype(int)*-1
home = cb[cb['team']==cb['home_t']]

#RUN THIS CELL, THEN PREP WORK DONE (DONT RUN AGAIN!) --> ALL OF YOUR DIFFERENCES WILL BE 0 at first!!
home_initial = pd.DataFrame()
away_initial = pd.DataFrame()
away_initial[['initial_lines','team']] = away[['lines','team']]
home_initial[['initial_lines','team']] = home[['lines','team']]

both = pd.DataFrame()
both['away_team'] = list(away_initial['team'])
both['home_team'] = list(home_initial['team'])
both['away_line'] = list(away_initial['initial_lines'])
both['home_line'] = list(home_initial['initial_lines'])

both.to_csv(f'{PATH}CBB_1_31.csv',index = False)

both.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,away_team,home_team,away_line,home_line
0,Fresno State,Coastal Carolina,-3,3.0
1,Xavier,Texas A&M,4,-4.0
2,Villanova,Kansas,4,-4.5
3,North Carolina,Duke,4,-4.0


In [5]:
#Getting email + credential deets
#to get it to work for your g

#mail account (https://support.google.com/accounts/answer/6010255#zippy=%2Cif-less-secure-app-access-is-on-for-your-account), click on 'less secure app access', click allow
SENDER_EMAIL = 'williamc@ggny.com'
SENDER_PASS = 'William112399@'
REC_EMAIL = 'wac6er@virginia.edu'

#in minutes, time you want to monitor
time_mins = 1000

Saving a table that tracks in game line differences for NCAAB + emailing an alert if games lines change by xx

In [6]:
for i in range(1000):
  all_games_today_away = pd.read_csv((f'{PATH}away_games.csv'))
  
  ff = all_games_today_away.iloc[:,-6:-4]
  ff['time'] = all_games_today_away['time_from_start']
  ff['difference'] = all_games_today_away['difference']
  ff['team'] = all_games_today_away['team']

  check = ff[ff['difference']<-10]

  if len(check) > 1:
    mail_content_away = check.to_string()


    #credentials
    sender_address = SENDER_EMAIL
    sender_pass = SENDER_PASS
    receiver_address = REC_EMAIL
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address

    #subject + i to make it new email (will be changed to range of loop)
    message['Subject'] = '!!hammer time!!' + str(i)

    #random
    message.attach(MIMEText(mail_content_away, 'plain'))
    session = smtplib.SMTP('smtp.gmail.com', 587) 
    session.starttls()
    session.login(sender_address, sender_pass) 
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()

    all_games_today_home = pd.read_csv((f'{PATH}home_games.csv'))

    ff = all_games_today_home.iloc[:,-6:-4]
    ff['time'] = all_games_today_home['time_from_start']
    ff['difference'] = all_games_today_home['difference']
    ff['team'] = all_games_today_home['team']

    check = ff[ff['difference']<-1]

  if len(check) > 1:
    mail_content_home = check.to_string()


    #credentials
    sender_address = SENDER_EMAIL
    sender_pass = SENDER_PASS
    receiver_address = REC_EMAIL
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address

    #subject + i to make it new email (will be changed to range of loop)
    message['Subject'] = '!!hammer time!!' + str(i)

    #random
    message.attach(MIMEText(mail_content_home, 'plain'))
    session = smtplib.SMTP('smtp.gmail.com', 587) 
    session.starttls()
    session.login(sender_address, sender_pass) 
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()

  time.sleep(random.uniform(30,60))

KeyboardInterrupt: ignored